## 0. Info & Summary
작성일 : 211112 / 작성자 : 김선진

## 1. 아이템 정의
DSP300 을 이해 바탕으로 Linear, Ridge, Lasso, ElasticNet 으로 구현하여 표현 

## 2. 목적
모델 개발 방향 및 주요 인자 search

----------------------------------
## 3. 방법 및 코드

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import os, warnings, shutil
import warnings, random, openpyxl
warnings.filterwarnings(action='ignore')  

from tqdm import tqdm

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42)
pd.set_option('max_columns', 130, 'max_rows', 250)



In [2]:
raw_train_df = pd.read_pickle("C:\\Users\\OnePredict\\Desktop\\SEMES_work\\lotus_211111_all-train-test-dataset\\lotus_train_df.pkl")
raw_test_df = pd.read_pickle("C:\\Users\\OnePredict\\Desktop\\SEMES_work\\lotus_211111_all-train-test-dataset\\lotus_test_df.pkl")
print(raw_train_df.shape, raw_test_df.shape)
display(raw_train_df.head(), raw_test_df.head())

(70609, 940) (4963, 940)


,log_number,state,Date,esu_PROC_RCP_STEP,esu_RPM,esu_Bowl_Exh_R,esu_Double_Feature_SCANCOUNT,esu_IPA_SCANCOUNT,esu_MRNozzle_SCANCOUNT,esu_Chuck_Bowl_Sh1_FLOW,esu_BACK_DI_FLOW,esu_BACK_N2_FLOW,esu_BACK_HOTDI_FLOW,esu_AIR_SUCTION_FLOW,esu_CHM_EXHAUST_FLOW,esu_BACKSIDE_CAP_N2_FLOW,esu_FINAL_DIW_FLOW,esu_CHM_DHF_FLOW,esu_CHM_DSP300_FLOW,esu_BACK_SC1_FLOW,esu_BACK_DHF_FLOW,esu_BACK_DSP300_FLOW,esu_AEROSOL_N2_FLOW,esu_AEROSOL_DI_FLOW,esu_CHM_SC1_DF_FLOW,esu_IPA_N2_FLOW,esu_HOT_IPA_FLOW,esu_HIPA HICV IN,esu_HIPA HICV OUT,esu_DIW1_FLOW,esu_DIW2_FLOW,esu_SSONICSWERVE,esu_WAFERSAFETY,esu_Environ_Exh,esu_SEL_THETA_POS1,esu_SEL_THETA_POS2,esu_SEL_THETA_POS3,esu_SEL_THETA_VEL1,esu_SEL_THETA_VEL2,esu_SEL_THETA_VEL3,esu_Double_Feature_THETA_POS,esu_Double_Feature_THETA_VEL,esu_Double_Feature_Z_POS,esu_IPA_THETA_POS,esu_IPA_THETA_VEL,esu_IPA_Z_POS,esu_MRNozzle_THETA_POS,esu_MRNozzle_THETA_VEL,esu_MRNozzle_Z_POS,esu_Hepa,esu_FLOW_SWITCH1,esu_FLOW_SWITCH2,esu_FLOW_SWITCH3,esu_CHAMBER_DOOR_STATUS,esu_SOUND_WAVE,esu_WAFER_TEMP,esu_HEPA_OUTPUT,esu_CDA_HEIGHT,esu_CDA_PRESS,esu_PIEZO_DRIVER_SETMODE,esu_HPU_PRESSURE,esu_REAL_BOWL_POS_STATUS,esu_AUTO_DAMPER_DEGREE_INFO1,esu_AUTO_DAMPER_DEGREE_INFO2,esu_CHAMBER_TEMPERTURE,...,tank10_Sub2_Density_Check,tank10_Main_Density_Check,tank10_Dnst1OfH2O2_Tank,tank10_Dnst1OfOther_Tank,tank10_Dnst2OfH2O2_Tank,tank10_Dnst2OfOther_Tank,tank10_DnstOfH2O2_Supply,tank10_DnstOfOther_Supply,tank10_RateOfACT,tank10_RateOfH2O2,tank10_RateOfDIW,tank10_TempOfMixHeater1Set,tank10_TempOfMixHeater1Act,tank10_TempOfMixHeater2Set,tank10_TempOfMixHeater2Act,tank10_TempOfSupplyHeaterSet,tank10_TempOfSupplyHeaterAct,tank10_TempOfMixtank1,tank10_TempOfMixtank2,tank10_TempOfMixtank3,tank10_TempOfReturnAct,tank10_LifeTimeSet,tank10_LifeTime,tank10_LifeCount,tank10_MainphMeter,tank10_FlowOfSub1ConPCW,tank10_FlowOfSub2ConPCW,tank10_FlowOfMainConPCW,tank10_PressOfPump,tank10_PressOfSupply,tank10_FlowOfSupply,tank10_FlowOfPCW,tank10_Exhaust,tank10_SetFlowOfACT,tank10_SetFlowOfH2O2,tank10_SetFlowOfCDIW,tank10_SetFlowOfHDIW,tank10_CurFlowOfACT,tank10_CurFlowOfH2O2,tank10_CurFlowOfCDIW,tank10_CurFlowOfHDIW,tank10_FlowOfACT,tank10_FlowOfH2O2,tank10_FlowOfCDIW,tank10_FlowOfHDIW,tank10_ACT_IN_PRESS,tank10_H2O2_IN_PRESS,tank10_Sub1phMeter,tank10_Sub2phMeter,tank10_PressOfRecycle_Filter_In,tank10_PressOfRecycle_Filter_Out,tank10_PressOfFilter_Out,tank10_CurrentOutputOfMixHeater,tank10_StateOfMixHeaterFullPower,tank10_StateOfSubCirPump,tank10_PressOfSubCirPumpInAir,tank10_StateOfMainCirPump,tank10_PressOfMainCirPumpInAir,tank10_StateOfMainSubPumpExch,tank10_TempOfDnstChemSampling,tank10_StateOfDnstMeasuring,tank10_HalogenOfUV_DS,tank10_HalogenOfUV_DR,tank10_HalogenOfNIR_DS,tank10_HalogenOfNIR_DR
0,1261,normal,2021-08-10 11:33:29.000,0.0,0.0,199.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,145.0,0.0,500.0,501.0,500.0,497.0,1526.0,0.0,0.0,501.0,0.0,239.0,127.0,88.0,1133.0,988.0,0.0,0.0,70.0,-30398.0,0.0,0.0,0.0,0.0,0.0,-24549.0,0.0,-500.0,-24849.0,0.0,0.0,-32700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101.0,627.0,272.0,700.0,491.0,700.0,460.0,700.0,700.0,699.0,497.0,461.0,723.0,1440.0,0.0,0.0,0.0,0.0,284.0,288.0,762.0,306.0,123.0,0.0,17.0,484.0,3009.0,1305.0,0.0,487.0,3012.0,1329.0,0.0,0.0,0.0,0.0,0.0,105.0,127.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1261,normal,2021-08-10 11:33:29.100,0.0,0.0,199.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,145.0,0.0,500.0,499.0,500.0,502.0,1526.0,0.0,0.0,501.0,0.0,239.0,127.0,88.0,1154.0,996.0,0.0,0.0,70.0,-30398.0,0.0,0.0,0.0,0.0,0.0,-24549.0,0.0,-500.0,-24849.0,0.0,0.0,-32700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101.0,627.0,272.0,700.0,491.0,700.0,459.9,700.0,699.9,699.0,497.0,461.0,723.0,1440.0,0.0,0.0,0.0,0.0,284.0,288.1,754.4,302.3,123.4,0.0,17.1,484.0,3009.0,1305.0,0.0,487.0,3012.0,1329.0,0.0,0.0,0.0,0.0,0.0,105.0,127.1,0.0,0.0,0.0,0.0,0.0,

,log_number,state,Date,esu_PROC_RCP_STEP,esu_RPM,esu_Bowl_Exh_R,esu_Double_Feature_SCANCOUNT,esu_IPA_SCANCOUNT,esu_MRNozzle_SCANCOUNT,esu_Chuck_Bowl_Sh1_FLOW,esu_BACK_DI_FLOW,esu_BACK_N2_FLOW,esu_BACK_HOTDI_FLOW,esu_AIR_SUCTION_FLOW,esu_CHM_EXHAUST_FLOW,esu_BACKSIDE_CAP_N2_FLOW,esu_FINAL_DIW_FLOW,esu_CHM_DHF_FLOW,esu_CHM_DSP300_FLOW,esu_BACK_SC1_FLOW,esu_BACK_DHF_FLOW,esu_BACK_DSP300_FLOW,esu_AEROSOL_N2_FLOW,esu_AEROSOL_DI_FLOW,esu_CHM_SC1_DF_FLOW,esu_IPA_N2_FLOW,esu_HOT_IPA_FLOW,esu_HIPA HICV IN,esu_HIPA HICV OUT,esu_DIW1_FLOW,esu_DIW2_FLOW,esu_SSONICSWERVE,esu_WAFERSAFETY,esu_Environ_Exh,esu_SEL_THETA_POS1,esu_SEL_THETA_POS2,esu_SEL_THETA_POS3,esu_SEL_THETA_VEL1,esu_SEL_THETA_VEL2,esu_SEL_THETA_VEL3,esu_Double_Feature_THETA_POS,esu_Double_Feature_THETA_VEL,esu_Double_Feature_Z_POS,esu_IPA_THETA_POS,esu_IPA_THETA_VEL,esu_IPA_Z_POS,esu_MRNozzle_THETA_POS,esu_MRNozzle_THETA_VEL,esu_MRNozzle_Z_POS,esu_Hepa,esu_FLOW_SWITCH1,esu_FLOW_SWITCH2,esu_FLOW_SWITCH3,esu_CHAMBER_DOOR_STATUS,esu_SOUND_WAVE,esu_WAFER_TEMP,esu_HEPA_OUTPUT,esu_CDA_HEIGHT,esu_CDA_PRESS,esu_PIEZO_DRIVER_SETMODE,esu_HPU_PRESSURE,esu_REAL_BOWL_POS_STATUS,esu_AUTO_DAMPER_DEGREE_INFO1,esu_AUTO_DAMPER_DEGREE_INFO2,esu_CHAMBER_TEMPERTURE,...,tank10_Sub2_Density_Check,tank10_Main_Density_Check,tank10_Dnst1OfH2O2_Tank,tank10_Dnst1OfOther_Tank,tank10_Dnst2OfH2O2_Tank,tank10_Dnst2OfOther_Tank,tank10_DnstOfH2O2_Supply,tank10_DnstOfOther_Supply,tank10_RateOfACT,tank10_RateOfH2O2,tank10_RateOfDIW,tank10_TempOfMixHeater1Set,tank10_TempOfMixHeater1Act,tank10_TempOfMixHeater2Set,tank10_TempOfMixHeater2Act,tank10_TempOfSupplyHeaterSet,tank10_TempOfSupplyHeaterAct,tank10_TempOfMixtank1,tank10_TempOfMixtank2,tank10_TempOfMixtank3,tank10_TempOfReturnAct,tank10_LifeTimeSet,tank10_LifeTime,tank10_LifeCount,tank10_MainphMeter,tank10_FlowOfSub1ConPCW,tank10_FlowOfSub2ConPCW,tank10_FlowOfMainConPCW,tank10_PressOfPump,tank10_PressOfSupply,tank10_FlowOfSupply,tank10_FlowOfPCW,tank10_Exhaust,tank10_SetFlowOfACT,tank10_SetFlowOfH2O2,tank10_SetFlowOfCDIW,tank10_SetFlowOfHDIW,tank10_CurFlowOfACT,tank10_CurFlowOfH2O2,tank10_CurFlowOfCDIW,tank10_CurFlowOfHDIW,tank10_FlowOfACT,tank10_FlowOfH2O2,tank10_FlowOfCDIW,tank10_FlowOfHDIW,tank10_ACT_IN_PRESS,tank10_H2O2_IN_PRESS,tank10_Sub1phMeter,tank10_Sub2phMeter,tank10_PressOfRecycle_Filter_In,tank10_PressOfRecycle_Filter_Out,tank10_PressOfFilter_Out,tank10_CurrentOutputOfMixHeater,tank10_StateOfMixHeaterFullPower,tank10_StateOfSubCirPump,tank10_PressOfSubCirPumpInAir,tank10_StateOfMainCirPump,tank10_PressOfMainCirPumpInAir,tank10_StateOfMainSubPumpExch,tank10_TempOfDnstChemSampling,tank10_StateOfDnstMeasuring,tank10_HalogenOfUV_DS,tank10_HalogenOfUV_DR,tank10_HalogenOfNIR_DS,tank10_HalogenOfNIR_DR
0,1275,diff_normal,2021-08-10 11:57:03.000,0.0,0.0,199.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,145.0,0.0,501.0,503.0,501.0,501.0,1526.0,0.0,0.0,501.0,0.0,236.0,128.0,89.0,0.0,0.0,0.0,0.0,70.0,-30398.0,0.0,0.0,0.0,0.0,0.0,-24549.0,0.0,-500.0,-24849.0,0.0,0.0,-32700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101.0,627.0,272.0,700.0,491.0,700.0,459.0,700.0,698.0,699.0,497.0,462.0,725.0,1440.0,0.0,0.0,0.0,0.0,285.0,289.0,719.0,274.0,125.0,0.0,18.0,484.0,3009.0,1305.0,0.0,487.0,3012.0,1329.0,0.0,0.0,0.0,0.0,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1275,diff_normal,2021-08-10 11:57:03.100,0.0,0.0,199.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,145.0,0.0,501.0,503.0,501.0,501.0,1526.0,0.0,0.0,501.0,0.0,236.0,128.0,89.0,0.0,0.0,0.0,0.0,71.0,-30398.0,0.0,0.0,0.0,0.0,0.0,-24549.0,0.0,-500.0,-24849.0,0.0,0.0,-32700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101.0,627.0,272.0,700.0,491.0,700.0,459.0,700.0,698.0,699.0,497.0,462.0,725.0,1440.0,0.0,0.0,0.0,0.0,285.0,289.0,719.0,274.0,125.0,0.0,18.0,484.0,3009.0,1305.0,0.0,487.0,3012.0,1329.0,0.0,0.0,0.0,0.0,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,

In [3]:
import chart_studio
chart_studio.tools.set_credentials_file(username='akangel0307', api_key='1lYJMFLLzvtiCm4LXSUn')
from chart_studio.plotly import plot, iplot

import plotly.graph_objects as go


In [4]:
raw_ttl_df = pd.concat([raw_train_df, raw_test_df],0).sort_values(['Date'],ascending=True).reset_index(drop=True)
raw_ttl_df.Date.value_counts()

2021-08-10 11:33:29.000    1
2021-08-10 13:26:27.000    1
2021-08-10 13:26:28.600    1
2021-08-10 13:26:28.500    1
2021-08-10 13:26:28.400    1
                          ..
2021-08-10 12:16:10.200    1
2021-08-10 12:16:10.100    1
2021-08-10 12:16:10.000    1
2021-08-10 12:16:09.900    1
2021-08-11 09:03:28.500    1
Name: Date, Length: 75572, dtype: int64

In [5]:
[i for i in X_train if "main" in i]

NameError: name 'X_train' is not defined

## test, train set 정리

In [ ]:
# 절때 쓰면 안되는 column 모음 
never_use_column = [
    'Date',
    'esu_CHM_DSP300_FLOW',
    'esu_CHM_DSP300_APAC_Sensor_Data',
    'esu_BACK_DSP300_FLOW',
    'esu_BACK_DSP300_APAC_Sensor_Data',
    'state',
    'log_number',
    'esu_CHM_DSP300_APAC_Current',
    "esu_BACK_DSP300_APAC_Current",
    "tank4_DATA_SEQ",
    'mainio_DATA_SEQ'
]
after_check = [ 
    'mainio_FP1_Oxygen_Density',
    'esu_Chuck_Bowl_Sh1_FLOW_DispenseState',
    'esu_BACK_DI_FLOW_DispenseState',
    'esu_ExtensionalBowlStage',
    # 'esu_CHM_DSP300_FLOW_TargetFlow',
    'esu_BACK_DSP300_FLOW_TargetFlow',
    'esu_REAL_BOWL_POS_STATUS',
    'esu_Bowl_Exh_L',
    'esu_CHM_DSP300_FLOW_TargetFlow'
    ]
### main io 조절 ######
mainio = [i for i in raw_ttl_df if "main" in i]


### tank io 조절 ######
tank_4 = [i for i in raw_ttl_df if "tank4" in i]
tank = [i for i in raw_ttl_df if "tank" in i]
not_use_tank = [i for i in tank if i not in tank_4]

### esu io 조절 ###
dispensestate = [i for i in raw_ttl_df if "DispenseState" in i]
esu_column = [i for i in raw_ttl_df if "esu" in i]
esu_column_nondis = [i for i in esu_column if i not in dispensestate]


X_train = raw_train_df.drop(never_use_column+not_use_tank+after_check+esu_column_nondis, axis=1)
X_test = raw_test_df.drop(never_use_column+not_use_tank+after_check+esu_column_nondis, axis=1)

y_train = raw_train_df["esu_CHM_DSP300_FLOW"]
y_test = raw_test_df["esu_CHM_DSP300_FLOW"]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(70609, 285) (4963, 285) (70609,) (4963,)


In [6]:
[i for i in raw_train_df if "DI" in i]

['esu_BACK_DI_FLOW',
 'esu_BACK_HOTDI_FLOW',
 'esu_FINAL_DIW_FLOW',
 'esu_AEROSOL_DI_FLOW',
 'esu_DIW1_FLOW',
 'esu_DIW2_FLOW',
 'esu_CHAMBER_HUMIDITY',
 'esu_CHAMBER_DIFF_PRESS',
 'esu_BACK_HOTDI_FlowSwitch',
 'esu_BACK_DI_FLOW_TargetFlow',
 'esu_BACK_DI_FLOW_CtrlValue',
 'esu_BACK_HOTDI_FLOW_TargetFlow',
 'esu_BACK_HOTDI_FLOW_CtrlValue',
 'esu_FINAL_DIW_FLOW_TargetFlow',
 'esu_FINAL_DIW_FLOW_CtrlValue',
 'esu_AEROSOL_DI_FLOW_TargetFlow',
 'esu_AEROSOL_DI_FLOW_CtrlValue',
 'esu_DIW1_FLOW_TargetFlow',
 'esu_DIW1_FLOW_CtrlValue',
 'esu_DIW2_FLOW_TargetFlow',
 'esu_DIW2_FLOW_CtrlValue',
 'esu_BACK_DI_FLOW_DispenseState',
 'esu_BACK_HOTDI_FLOW_DispenseState',
 'esu_FINAL_DIW_FLOW_DispenseState',
 'esu_AEROSOL_DI_FLOW_DispenseState',
 'esu_DIW1_FLOW_DispenseState',
 'esu_DIW2_FLOW_DispenseState',
 'esu_BACK_DI_APAC_Sensor_Data',
 'esu_BACK_DI_APAC_Current',
 'esu_BACK_HOTDI_APAC_Sensor_Data',
 'esu_BACK_HOTDI_APAC_Current',
 'esu_FINAL_DIW_APAC_Sensor_Data',
 'esu_FINAL_DIW_APAC_Current',


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=raw_train_df["esu_CHM_DSP300_FLOW_TargetFlow"], 
                         name = "DSP300_Target",
                         line = dict(color = "rgba(0,150,0,0.4)", width=2)))
fig.add_trace(go.Scatter(y=raw_train_df["esu_CHM_DSP300_FLOW"], 
                         name = "DSP300_Flow",
                         line = dict(color = "rgba(255,0,0,0.4)", width=2)))
# fig.add_trace(go.Scatter(y=raw_train_df["esu_DIW1_FLOW"], name = "DI water"))
# fig.add_trace(go.Scatter(y=raw_train_df[""]))
fig.update_layout(template="plotly_white", 
                  title="Clean Flow process",)
fig.show()

## linear Regression 사용 

In [ ]:
from sklearn import linear_model
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, r2_score

linear_regr = linear_model.LinearRegression()

linear_regr.fit(X_train, y_train)
linear_regr_pred = linear_regr.predict(X_test)
rmse_linear = np.sqrt(np.mean((y_test - linear_regr_pred)**2))

# print("Coefficients: \n",linear_regr.coef_)
print("mean_squared_error: %.2f" %mean_squared_error(y_test, linear_regr_pred))
print("Coefficient of determination: %.2f" %r2_score(y_test, linear_regr_pred))
print("RMSE: %.2f" %rmse_linear)

mean_squared_error: 2087.93
Coefficient of determination: 0.90
RMSE: 45.69


In [ ]:
# from sklearn.preprocessing import PolynomialFeatures #TODO Polynomial 과 linear 의 차이가 무엇인지?

# poly = PolynomialFeatures(degree=100)
# X_poly = poly.fit_transform(X_train)

# poly.fit(X_poly, y_train)
# lin2 = LinearRegression()
# lin2.fit(X_poly, y_train)
# pred_lin2 = lin2.predict(X_test)
# rmse = np.sqrt(np.mean((y_test-pred_lin2)**2))

# print("RMSE: %.2f" %rmse)

## Ridge Regression 사용 


In [ ]:
from sklearn.linear_model import Ridge

# alpha 값을 케이스 별로 나눔 
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]

Ridge_model = Ridge(alpha = 1e-4 ,max_iter=100)
Ridge_model.fit(X_train, y_train)
Ridge_model_pred = Ridge_model.predict(X_test)

rmse_Ridge = np.sqrt(np.mean((y_test - Ridge_model_pred)**2))

print("RMSE: %.2f" %rmse_Ridge)


RMSE: 45.71


## Lasso Regression 사용 

In [ ]:
from sklearn.linear_model import Lasso

Lasso_model = Lasso(alpha =1e-4, max_iter=100)
Lasso_model.fit(X_train, y_train)
Lasso_model_pred = Lasso_model.predict(X_test)

rmse_Lasso = np.sqrt(np.mean((y_test - Lasso_model_pred)**2))
print("RMSE: %.2f" %rmse_Lasso)

RMSE: 45.65


## ElasticNet Regression 사용

In [ ]:
from sklearn.linear_model import ElasticNet

ElasticNet_model = ElasticNet(alpha=1e-4, l1_ratio=0.5)
ElasticNet_model.fit(X_train,y_train)
ElasticNet_model_pred = ElasticNet_model.predict(X_test)

rmse_ElasticNet = np.sqrt(np.mean((ElasticNet_model_pred-y_test)**2))
print("RMSE: %.2f" %rmse_ElasticNet)

RMSE: 45.60


## Regression 들 figure 뽑기 

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test, name="test"))
fig.add_trace(go.Scatter(y=linear_regr_pred, name = "Linear_pred[RMSE : %.2f]"%rmse_linear, line=dict(color='rgba(255, 0, 0, 1)')))
fig.add_trace(go.Scatter(y=Ridge_model_pred, name = "Ridge_pred[RMSE : %.2f]"%rmse_Ridge, line=dict(color='rgba(255, 125, 0, 1)')))
fig.add_trace(go.Scatter(y=Lasso_model_pred, name = "Lasso_pred[RMSE : %.2f]"%rmse_Lasso, line=dict(color='rgba(255, 0, 255, 1)')))
fig.add_trace(go.Scatter(y=ElasticNet_model_pred, name = "ElasticNet_pred[RMSE : %.2f]"%rmse_ElasticNet, line=dict(color='rgba(125,0, 125, 1)')))

fig.update_layout(title = "Regression", template = "plotly_white")
fig.show()
plot(fig, filename = 'test', auto_open=True)

'https://plotly.com/~akangel0307/3/'

## alpha에 따른 변화 

In [ ]:
# from sklearn.linear_model import Lasso

# Lasso_model = Lasso(alpha =100, max_iter=100)
# Lasso_model.fit(X_train, y_train)
# Lasso_model_pred = Lasso_model.predict(X_test)
# rmse_Lasso = np.sqrt(np.mean((y_test - Lasso_model_pred)**2))
# print("RMSE: %.2f" %rmse_Lasso)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, max_depth=50, random_state=77)
model.fit(X_train, y_train)
pred = model.predict(X_test)
rmse = np.sqrt(np.mean((y_test - pred)**2))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test))
fig.add_trace(go.Scatter(y=pred))
fig.show()

## importance 구하기

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test, name="test"))
fig.add_trace(go.Scatter(y=linear_regr_pred, name = "Lasso_pred_alpha=1e-10[RMSE : %.2f]"%rmse_linear))
fig.add_trace(go.Scatter(y=Ridge_model_pred, name = "Lasso_pred_alpha=1e-5[RMSE : %.2f]"%rmse_Ridge))
fig.add_trace(go.Scatter(y=Lasso_model_pred, name = "Lasso_pred_alpha=1e+5[RMSE : %.2f]"%rmse_Lasso))
fig.add_trace(go.Scatter(y=ElasticNet_model_pred, name = "Lasso_pred_alpha=1e+10[RMSE : %.2f]"%rmse_ElasticNet))
fig.update_layout(title = "Regression")
fig.show()

In [ ]:
import eli5 
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(linear_regr, random_state=77).fit(X_test, y_test)
eli5.show_weights(perm,top=5,feature_names = X_test.columns.tolist())

# print(eli5.format_as_text(eli5.explain_weights(perm, feature_names = X_test.columns.tolist())))

In [ ]:
perm_Ridge = PermutationImportance(Ridge_model, random_state=77).fit(X_test, y_test)
eli5.show_weights(perm_Ridge,top=5,feature_names = X_test.columns.tolist())

In [ ]:
perm_Lasso = PermutationImportance(Lasso_model, random_state=77).fit(X_test, y_test)
eli5.show_weights(perm_Lasso,top=5,feature_names = X_test.columns.tolist())

In [ ]:
perm_ElasticNet = PermutationImportance(ElasticNet_model, random_state=77).fit(X_test, y_test)
eli5.show_weights(perm_ElasticNet,top=5,feature_names = X_test.columns.tolist())

## 4. 결론
#### - Linear , Ridge, Lasso, ElasticNet 결과, 뽑히는 Feature의 퀄리티 차이가 큼. 
#### - 규제가 생길수록 관련도 정의가 깔끔해지지만, 결과는 똑같기 때문에 너무 간단한 모델로는 DSP300을 추정할 수 없음.
